# 1125

## Hello C++

In [1]:
%%bash
cat > hello.cpp << CPPCODE

#include <iostream>
using namespace std;
int main() {
    cout << "hello" << endl;
}

CPPCODE
g++-14 -std=c++26 -Wall -Wextra -Wpedantic -Wshadow -Wconversion hello.cpp -o h && ./h

hello


## Hello Python

In [2]:
%%bash
cat > hello.py << PYCODE

print("Hello World")

PYCODE

In [3]:
!python3 hello.py

Hello World


## Hello Cython

In [4]:
%%bash
cat > setup.py << PYCODE

from setuptools import setup
from Cython.Build import cythonize
setup(
    ext_modules = cythonize("hello.py")
)

PYCODE

In [5]:
!python setup.py build_ext --inplace

Compiling hello.py because it changed.
[1/1] Cythonizing hello.py
running build_ext
building 'hello' extension
gcc -fno-strict-overflow -Wsign-compare -DNDEBUG -O3 -Wall -fPIC -O3 -isystem /home/simple/miniforge3/include -fPIC -O3 -isystem /home/simple/miniforge3/include -fPIC -I/home/simple/1125/simplenv/include -I/home/simple/miniforge3/include/python3.12 -c hello.c -o build/temp.linux-aarch64-cpython-312/hello.o
gcc -shared -Wl,--allow-shlib-undefined -Wl,-rpath,/home/simple/miniforge3/lib -Wl,-rpath-link,/home/simple/miniforge3/lib -L/home/simple/miniforge3/lib -Wl,--allow-shlib-undefined -Wl,-rpath,/home/simple/miniforge3/lib -Wl,-rpath-link,/home/simple/miniforge3/lib -L/home/simple/miniforge3/lib build/temp.linux-aarch64-cpython-312/hello.o -o build/lib.linux-aarch64-cpython-312/hello.cpython-312-aarch64-linux-gnu.so
copying build/lib.linux-aarch64-cpython-312/hello.cpython-312-aarch64-linux-gnu.so -> 


In [6]:
import hello

Hello World


## Hello Cell Magic

In [7]:
%load_ext Cython

In [8]:
%%cython
cpdef int add(int a, int b):
    return a + b + 1

In [9]:
add(2, 3)

6

## Hello Fibonacci

In [10]:
def fib(n):
    """Print the Fibonacci series up to n."""
    if n <= 1: return 1
    return fib(n-1) + fib(n-2)

In [11]:
fib(1), fib(2)

(1, 2)

In [12]:
%%timeit

fib(10)

6.34 μs ± 323 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)


In [13]:
%%cython
cpdef int fibo(int n):
    if n <= 1: return 1
    return fibo(n-1) + fibo(n-2)

In [14]:
fibo(1), fibo(2)

(1, 2)

In [15]:
%%timeit

fibo(10)

143 ns ± 3.66 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


## Cython Include C

In [16]:
%%bash
cat > fibon.hpp << CPPCODE

#pragma once
long long fibon(int n);

CPPCODE

In [17]:
%%bash

cat > fibon.hpp << hPPCODE
    #pragma once
    long long fibon(int n);
hPPCODE
cat > fibon.cpp << CPPCODE
    long long fibon(int n) {
        if (n <= 1) return 1;
        return fibon(n - 1) + fibon(n - 2);
    }
CPPCODE
g++-14 -std=c++26 -O3 -fPIC -shared fibon.cpp -o libfibon.so

In [18]:
%%bash
ls libfibon.so fibon.hpp

fibon.hpp
libfibon.so


In [19]:
%%cython --cplus -I. -L. -lfibon --link-args=-Wl,-rpath,.

# C++ mode, include + lib dirs = current directory
# --link-args passes extra_link_args to the linker

cdef extern from "fibon.hpp":
    long long fibon(int n)

def fibon_py(int n):
    """
    Python wrapper around C++ fibon(int n).
    """
    return fibon(n)

In [20]:
%%timeit

fibon_py(10)

88.4 ns ± 2.73 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)
